In [11]:
import numpy as np
from sklearn.cluster import MiniBatchKMeans
import matplotlib.pyplot as plt


from PIL import Image, ImageDraw, ImageOps
from sklearn import datasets
from sklearn.preprocessing import scale

In [2]:
class Tools:
    @staticmethod
    def drawCentroids(centroids='ro'):
        plt.plot(centroids[:, 0], centroids[:, 1], 'ro')

    @staticmethod
    def drawCentroid(centroid, symbol='ro'):
        plt.plot(centroid[0], centroid[1], symbol)

    @staticmethod
    def drawDataSet(X, symbol):
        plt.plot(X[:, 0], X[:, 1], symbol)

    @staticmethod
    def distBetween(a, b):
        return np.sqrt(np.power(a[0]-b[0], 2) + np.power(a[1] - b[1], 2))

    @staticmethod
    def generateSomeData(start, end, numbers):
        pass

    @staticmethod
    def exchange_binary_image_white_black(img):
        # img is an binary img
        img = img.convert('L')
        pixels = np.array(img)
        threshold = 100

        rows = len(pixels)
        cols = len(pixels[0])

        for i in range(rows):
            for j in range(cols):
                if pixels[i][j] > threshold:
                    pixels[i][j] = 0
                else:
                    pixels[i][j] = 255

        return pixels

In [8]:
# My KMeans class wrapper
class Cluster:
    def __init__(self, dataSet, centroid):
        self.dataSet = dataSet
        self.centroid = centroid

    def getCentroid(self):
        return self.centroid

    def getCluster(self):
        return self.dataSet


class KMeans:
    def __init__(self):
        self.centroids = np.array([])
        self.cluster_0 = []
        self.cluster_1 = []
        self.c_0 = None
        self.c_1 = None

    def split(self, dataSet):
        kM = MiniBatchKMeans(n_clusters=2)
        result = kM.fit(dataSet)

        self.c_0 = result.cluster_centers_[0]
        self.c_1 = result.cluster_centers_[1]

        index = 0
        for label in result.labels_:
            if label == 0:
                self.cluster_0.append(dataSet[index])
            else:
                self.cluster_1.append(dataSet[index])
            index += 1

        self.cluster_0 = np.array(self.cluster_0)
        self.cluster_1 = np.array(self.cluster_1)


In [9]:
# My GMeans
class GMeans:
    def __init__(self, strictLevel):
        self.strickLevel = strictLevel
        self.centroids = []
        self.cluster = []

    # recursive helper method
    def computeGMeans(self, X):
        centroid = np.mean(X, axis=0)

        km = KMeans()
        km.split(dataSet=X)
        v = km.c_0 - km.c_1

        X_prime = scale(X.dot(v) / (v.dot(v)))
        accept_split = GMeans.checkGaussianStatistic(X_prime, self.strickLevel)

        if accept_split:
            self.computeGMeans(km.cluster_0)
            self.computeGMeans(km.cluster_1)
        else:
            self.centroids.append(centroid)

    @staticmethod
    def checkGaussianStatistic(X, strictLevel):
        X = X - np.mean(X)
        output = anderson(X)
        statistic = output[0]
        critical_value = output[1][strictLevel]
        # print statistic, critical_value
        if statistic >= critical_value:
            return True
        else:
            return False

    def fit(self, X):
        self.computeGMeans(X)
        self.centroids = np.array(self.centroids)


In [12]:
# Experiment one:
# show the G-Mean works

 # demo one:
dataSet1 = np.random.randn(100, 2)+2
dataSet2 = np.random.randn(100, 2)+5
dataSet3 = np.random.randn(400, 2)-2
# dataSet4 = np.random.randn(50, 2)-5
dataSet5 = np.random.randn(20, 2)-10

total_dataSet = np.vstack((dataSet1, dataSet2))
total_dataSet = np.vstack((total_dataSet, dataSet3))
# total_dataSet = np.vstack((total_dataSet, dataSet4))
total_dataSet = np.vstack((total_dataSet, dataSet5))

total_dataSet = np.random.permutation(total_dataSet)

gM = GMeans(strictLevel=4)
gM.fit(total_dataSet)

print "found {} clusters".format(len(gM.centroids))

tl.drawDataSet(total_dataSet, 'g+')
tl.drawCentroids(gM.centroids)
plt.show()

NameError: global name 'anderson' is not defined